# node2vec
---
[node2vec](http://snap.stanford.edu/node2vec/) for link prediction:
1. Perform train-test split
1. Train skip-gram model on random walks within training graph
2. Get node embeddings from skip-gram model
3. Create bootstrapped edge embeddings by taking the Hadamard product of node embeddings
4. Train a logistic regression classifier on these edge embeddings (possible edge --> edge score between 0-1)
5. Evaluate these edge embeddings on the validation and test edge sets

node2vec source code: https://github.com/aditya-grover/node2vec

Dataset: The Facebook-like Forum Network was attained from the same online community as the online social network; however, the focus in this network is not on the private messages exchanged among users, but on users’ activity in the forum.

Number of nodes: 899
Number of edges: 7046
Number of time frames: 4

## 1. Read in Graph Data

In [9]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import scipy.sparse as sp
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import pickle

In [10]:
#EGO_USER = 0 # which ego network to look at

# Load pickled (adj, feat) tuple
#network_dir = './fb-processed/{0}-adj-feat.pkl'.format(EGO_USER)
#with open(network_dir, 'rb') as f:
#    adj, features = pickle.load(f)
    
#g = nx.Graph(adj) # re-create graph using node indices (0 to num_nodes-1)

Step 1: Load in the networks, make the training and test edge lists

In [11]:
MasterGraph = nx.read_edgelist("m_forum.csv", nodetype=int, delimiter=",")
for edge in MasterGraph.edges():
    MasterGraph[edge[0]][edge[1]]['weight'] = 1

print MasterGraph.number_of_nodes()
print MasterGraph.number_of_edges()

G1 = nx.read_edgelist("m1_forum.csv", nodetype = int, delimiter = ",")
for edge in G1.edges():
    G1[edge[0]][edge[1]]['weight'] = 1
G2 = nx.read_edgelist("m2_forum.csv", nodetype = int, delimiter = ",")
for edge in G2.edges():
    G2[edge[0]][edge[1]]['weight'] = 1
G3 = nx.read_edgelist("m3_forum.csv", nodetype = int, delimiter = ",")
for edge in G3.edges():
    G3[edge[0]][edge[1]]['weight'] = 1
G4 = nx.read_edgelist("m4_forum.csv", nodetype = int, delimiter = ",")
for edge in G4.edges():
    G4[edge[0]][edge[1]]['weight'] = 1
G13 = nx.read_edgelist("m4_forum.csv", nodetype = int, delimiter = ",")    
for edge in G13.edges():
    G13[edge[0]][edge[1]]['weight'] = 1

## All the nodes are in MasterNodes    
MasterNodes = MasterGraph.nodes()

899
7046


NameError: name 'G13' is not defined

In [ ]:
### Training - Test split  
'''''
first add all the nodes that are in MasterGraph but not in 
G4
'''''
for i in MasterNodes:
    if i not in G4.nodes():
        G4.add_node(i)
        
adj_sparse = nx.to_scipy_sparse_matrix(G4)

In [ ]:
print G4.number_of_nodes()
print G4.number_of_edges()
print G1.number_of_edges()
print G2.number_of_edges()
print G3.number_of_edges()

In [ ]:
from gae.preprocessing import mask_test_edges
np.random.seed(0) # make sure train-test split is consistent between notebooks

adj_sparse = nx.to_scipy_sparse_matrix(G4)

adj_train, train_edges, train_edges_false, val_edges, val_edges_false, \
    test_edges, test_edges_false = mask_test_edges(adj_sparse, test_frac=.3, val_frac=.0, prevent_disconnect = True)


In [ ]:
# Inspect train/test split
print "Total nodes:", adj_sparse.shape[0]
print "Total edges:", int(adj_sparse.nnz/2) # adj is symmetric, so nnz (num non-zero) = 2*num_edges
print "Training edges (positive):", len(train_edges)
print "Training edges (negative):", len(train_edges_false)
print "Validation edges (positive):", len(val_edges)
print "Validation edges (negative):", len(val_edges_false)
print "Test edges (positive):", len(test_edges)
print "Test edges (negative):", len(test_edges_false)

The positive training edges are in the edgelist "train_edges".

The negative training edges are in the edgelist "train_edges_false".

The positive test edges are in the edgelist "test_edges".

The negative test edges are in the edgelist "test_edges_false".

Step 2: add all the nodes in G1, G2, G3 that are not present

In [ ]:
'''
add all the nodes that are in the MasterGraph but not in 
G1, G2 and G3
'''
for i in MasterNodes:
    if i not in G1.nodes():
        G1.add_node(i)
    if i not in G2.nodes():
        G2.add_node(i)
    if i not in G3.nodes():
        G3.add_node(i)

In [ ]:
print "Edges before removal: "
print "G1:  ", G1.number_of_edges()
print "G2:  ", G2.number_of_edges()
print "G3:  ", G3.number_of_edges()


'''
for every snapshot, delete all the edges that occur in the 
test set, this is important because the training of node2vec
can only be done on the training network and not on edges that
are used for testing
'''
for i in range(0,len(test_edges)):
        if G1.has_edge(test_edges[i, 0], test_edges[i, 1]):
            G1.remove_edge(test_edges[i, 0], test_edges[i, 1])
        if G2.has_edge(test_edges[i, 0], test_edges[i, 1]):
            G2.remove_edge(test_edges[i, 0], test_edges[i, 1])
        if G3.has_edge(test_edges[i, 0], test_edges[i, 1]):
            G3.remove_edge(test_edges[i, 0], test_edges[i, 1])
            
print "Edges after removal: "
print "G1:  ", G1.number_of_edges()
print "G2:  ", G2.number_of_edges()
print "G3:  ", G3.number_of_edges()

## 3. Train node2vec (Learn Node Embeddings)

In [ ]:
import node2vec
from gensim.models import Word2Vec

In [ ]:
# node2vec settings
# NOTE: When p = q = 1, this is equivalent to DeepWalk

P = 1 # Return hyperparameter
Q = 1 # In-out hyperparameter
WINDOW_SIZE = 10 # Context size for optimization
NUM_WALKS = 10 # Number of walks per source
WALK_LENGTH = 80 # Length of walk per source
DIMENSIONS = 128 # Embedding dimension
DIRECTED = False # Graph directed/undirected
WORKERS = 8 # Num. parallel workers
ITER = 1 # SGD epochs

In [ ]:
# Preprocessing, generate walks
G1_n2v = node2vec.Graph(G1, DIRECTED, P, Q) # create node2vec graph instance
G2_n2v = node2vec.Graph(G2, DIRECTED, P, Q)
G3_n2v = node2vec.Graph(G3, DIRECTED, P, Q)

G1_n2v.preprocess_transition_probs()
G2_n2v.preprocess_transition_probs()
G3_n2v.preprocess_transition_probs()

walksG1 = G1_n2v.simulate_walks(NUM_WALKS, WALK_LENGTH)
walksG2 = G2_n2v.simulate_walks(NUM_WALKS, WALK_LENGTH)
walksG3 = G3_n2v.simulate_walks(NUM_WALKS, WALK_LENGTH)

walksG1 = [map(str, walk) for walk in walksG1]
walksG2 = [map(str, walk) for walk in walksG2]
walksG3 = [map(str, walk) for walk in walksG3]

# Train skip-gram model
modelG1 = Word2Vec(walksG1, size=DIMENSIONS, window=WINDOW_SIZE, min_count=0, sg=1, workers=WORKERS, iter=ITER)
modelG2 = Word2Vec(walksG2, size=DIMENSIONS, window=WINDOW_SIZE, min_count=0, sg=1, workers=WORKERS, iter=ITER)
modelG3 = Word2Vec(walksG3, size=DIMENSIONS, window=WINDOW_SIZE, min_count=0, sg=1, workers=WORKERS, iter=ITER)


# Store embeddings mapping
emb_mappingsG1 = modelG1.wv
emb_mappingsG2 = modelG2.wv
emb_mappingsG3 = modelG3.wv

## 4. Create Edge Embeddings

In [ ]:
# Create node embeddings matrix (rows = nodes, columns = embedding features)
emb_listG1 = []
emb_listG2 = []
emb_listG3 = []

for node_index in range(1, adj_sparse.shape[0]+1):
    node_str = str(node_index)
    
    node_embG1 = emb_mappingsG1[node_str]
    node_embG2 = emb_mappingsG2[node_str]
    node_embG3 = emb_mappingsG3[node_str]
    
    emb_listG1.append(node_embG1)
    emb_listG2.append(node_embG2)
    emb_listG3.append(node_embG3)
    
emb_matrixG1 = np.vstack(emb_listG1)
emb_matrixG2 = np.vstack(emb_listG2)
emb_matrixG3 = np.vstack(emb_listG3)

In [ ]:
# Generate bootstrapped edge embeddings (as is done in node2vec paper)
    # Edge embedding for (v1, v2) = hadamard product of node embeddings for v1, v2
def get_edge_embeddings(edge_list):
    embs = []
    for edge in edge_list:
        
        node1 = edge[0]
        node2 = edge[1]
        
        embG1_1 = emb_matrixG1[node1]
        embG1_2 = emb_matrixG1[node2]
        
        embG2_1 = emb_matrixG2[node1]
        embG2_2 = emb_matrixG2[node2]
        
        embG3_1 = emb_matrixG3[node1]
        embG3_2 = emb_matrixG3[node2]
        
        edge_embG1 = np.multiply(embG1_1, embG1_2)
        edge_embG2 = np.multiply(embG2_1, embG2_2)
        edge_embG3 = np.multiply(embG3_1, embG3_2)
        
        edge_emb = np.hstack((edge_embG1,edge_embG2, edge_embG3))
        embs.append(edge_emb)
        
    embs = np.array(embs)
    
    return embs

In [ ]:
# Train-set edge embeddings
pos_train_edge_embs = get_edge_embeddings(train_edges)
neg_train_edge_embs = get_edge_embeddings(train_edges_false)
train_edge_embs = np.concatenate([pos_train_edge_embs, neg_train_edge_embs])

# Create train-set edge labels: 1 = real edge, 0 = false edge
train_edge_labels = np.concatenate([np.ones(len(train_edges)), np.zeros(len(train_edges_false))])

# Val-set edge embeddings, labels
pos_val_edge_embs = get_edge_embeddings(val_edges)
neg_val_edge_embs = get_edge_embeddings(val_edges_false)
val_edge_embs = np.concatenate([pos_val_edge_embs, neg_val_edge_embs])
val_edge_labels = np.concatenate([np.ones(len(val_edges)), np.zeros(len(val_edges_false))])

# Test-set edge embeddings, labels
pos_test_edge_embs = get_edge_embeddings(test_edges)
neg_test_edge_embs = get_edge_embeddings(test_edges_false)
test_edge_embs = np.concatenate([pos_test_edge_embs, neg_test_edge_embs])

# Create val-set edge labels: 1 = real edge, 0 = false edge
test_edge_labels = np.concatenate([np.ones(len(test_edges)), np.zeros(len(test_edges_false))])

## 5. Evaluate Edge Embeddings

In [ ]:
# Train logistic regression classifier on train-set edge embeddings
from sklearn.linear_model import LogisticRegression
edge_classifier = LogisticRegression(random_state=0)
edge_classifier.fit(train_edge_embs, train_edge_labels)

In [ ]:
# Predicted edge scores: probability of being of class "1" (real edge)
# val_preds = edge_classifier.predict_proba(val_edge_embs)[:, 1]
# val_roc = roc_auc_score(val_edge_labels, val_preds)
# val_ap = average_precision_score(val_edge_labels, val_preds)

In [ ]:
# Predicted edge scores: probability of being of class "1" (real edge)
test_preds = edge_classifier.predict_proba(test_edge_embs)[:, 1]
test_roc = roc_auc_score(test_edge_labels, test_preds)
test_ap = average_precision_score(test_edge_labels, test_preds)

In [ ]:
# print 'node2vec Validation ROC score: ', str(val_roc)
# print 'node2vec Validation AP score: ', str(val_ap)
print 'node2vec Test ROC score: ', str(test_roc)
print 'node2vec Test AP score: ', str(test_ap)

# Baseline classifiers (Adamic Adar, Jaccard Index & Preferred Attachement)


In [ ]:
def get_roc_score(edges_pos, edges_neg, score_matrix):
    # Store positive edge predictions, actual values
    preds_pos = []
    pos = []
    for edge in edges_pos:
        preds_pos.append(score_matrix[edge[0], edge[1]]) # predicted score
        pos.append(adj_sparse[edge[0], edge[1]]) # actual value (1 for positive)
        
    # Store negative edge predictions, actual values
    preds_neg = []
    neg = []
    for edge in edges_neg:
        preds_neg.append(score_matrix[edge[0], edge[1]]) # predicted score
        neg.append(adj_sparse[edge[0], edge[1]]) # actual value (0 for negative)
        
    # Calculate scores
    preds_all = np.hstack([preds_pos, preds_neg])
    labels_all = np.hstack([np.ones(len(preds_pos)), np.zeros(len(preds_neg))])
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)
    return roc_score, ap_score

In [ ]:

    
adj_sparse = nx.to_scipy_sparse_matrix(MasterGraph)    
    
adj_train, train_edges, train_edges_false, val_edges, val_edges_false, \
    test_edges, test_edges_false = mask_test_edges(adj_sparse, test_frac=.3, val_frac=.00, prevent_disconnect=False)    
    
g_train = nx.from_scipy_sparse_matrix(adj_train)



## 3. Adamic-Adar

In [ ]:
# Compute Adamic-Adar indexes from g_train
aa_matrix = np.zeros(adj.shape)
for u, v, p in nx.adamic_adar_index(g_train): # (u, v) = node indices, p = Adamic-Adar index
    aa_matrix[u][v] = p
    aa_matrix[v][u] = p # make sure it's symmetric
    
# Normalize array
aa_matrix = aa_matrix / aa_matrix.max()

In [ ]:
# Calculate ROC AUC and Average Precision
aa_roc, aa_ap = get_roc_score(test_edges, test_edges_false, aa_matrix)

print 'Adamic-Adar Test ROC score: ', str(aa_roc)
print 'Adamic-Adar Test AP score: ', str(aa_ap)

## 4. Jaccard Coefficient

In [12]:
# Compute Jaccard Coefficients from g_train
jc_matrix = np.zeros(adj.shape)
for u, v, p in nx.jaccard_coefficient(g_train): # (u, v) = node indices, p = Jaccard coefficient
    jc_matrix[u][v] = p
    jc_matrix[v][u] = p # make sure it's symmetric
    
# Normalize array
jc_matrix = jc_matrix / jc_matrix.max()

NameError: name 'adj' is not defined

In [13]:
# Calculate ROC AUC and Average Precision
jc_roc, jc_ap = get_roc_score(test_edges, test_edges_false, jc_matrix)

print 'Jaccard Coefficient Test ROC score: ', str(jc_roc)
print 'Jaccard Coefficient Test AP score: ', str(jc_ap)

NameError: name 'test_edges' is not defined

In [14]:
# Calculate, store Adamic-Index scores in array
pa_matrix = np.zeros(adj.shape)
for u, v, p in nx.preferential_attachment(g_train): # (u, v) = node indices, p = Jaccard coefficient
    pa_matrix[u][v] = p
    pa_matrix[v][u] = p # make sure it's symmetric
    
# Normalize array
pa_matrix = pa_matrix / pa_matrix.max()

NameError: name 'adj' is not defined

In [15]:
# Calculate ROC AUC and Average Precision
pa_roc, pa_ap = get_roc_score(test_edges, test_edges_false, pa_matrix)

print 'Preferential Attachment Test ROC score: ', str(pa_roc)
print 'Preferential Attachment Test AP score: ', str(pa_ap)

NameError: name 'test_edges' is not defined